# Bernoulli Verification Example - Distributed Loads on a Console

Concentrated actions at the free end of a console for all dofs. Displacements of the free end, reaction dynams and internal dynams at the support are checked agains analytical solutions. The console has a length of $L$, tube section of diameter $d$ and thickness $t$ and an isotropic material definition.

In [156]:
from sigmaepsilon.solid.fem.cells import B3 as Beam
from sigmaepsilon.solid import BeamSection

# units in kN, cm

# geometry
L = 2500.

# material
Ex, nu = 210000.0, 0.3

# load
q = 1.0

# mesh
n = 25

# section
d, t = 500, 25
section = BeamSection('CHS', d=d, t=t, n=32)

# orientation


## Linear Solution

In [157]:
from sigmaepsilon.solid import Structure, LineMesh, PointData
from polymesh.space import StandardFrame, PointCloud, frames_of_lines
from polymesh.space.utils import index_of_closest_point
from polymesh.topo.tr import L2_to_L3
from linkeddeepdict import LinkedDeepDict
from linkeddeepdict.tools import getallfromkwargs
from neumann.linalg import linspace
from neumann.array import repeat
import numpy as np
import awkward as ak

iN = None  # index of the node at the free end

# section
section.calculate_section_properties()
section_props = section.section_properties
A, Ix, Iy, Iz = getallfromkwargs(['A', 'Ix', 'Iy', 'Iz'], **section_props)

# material
G = Ex / (2 * (1 + nu))
Hooke = np.array([
    [Ex*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, Ex*Iy, 0],
    [0, 0, 0, Ex*Iz]
])

nLoadCase = 3

# space
GlobalFrame = StandardFrame(dim=3)

# mesh
p0 = np.array([0., 0., 0.])
p1 = np.array([L, 0., 0])
coords = linspace(p0, p1, n+1)
coords = PointCloud(coords, frame=GlobalFrame).show()
topo = np.zeros((n, 2), dtype=int)
topo[:, 0] = np.arange(n)
topo[:, 1] = np.arange(n) + 1

if Beam.NNODE == 3:
    coords, topo = L2_to_L3(coords, topo)
iN = index_of_closest_point(coords, np.array([L, 0, 0]))

# essential boundary conditions
ebcinds = np.where(coords[:, 0] < 1e-12)[0]
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
fixity[ebcinds, :] = True

# natural boundary conditions
nodal_loads = np.zeros((coords.shape[0], 6, nLoadCase))
cell_loads = np.zeros((topo.shape[0], topo.shape[1], 6, nLoadCase))
cell_loads[:, :, 0, 0] = q

# pointdata
fixity = fixity.astype(float) * 1e12
pd = PointData(coords=coords, fixity=fixity, loads=nodal_loads,
               frame=GlobalFrame)

# celldata
frames = repeat(GlobalFrame.show(), topo.shape[0])
areas = np.full(topo.shape[0], section.A)
cd = Beam(topo=topo, loads=cell_loads, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)


In [158]:
structure.linsolve()
d = structure.nodal_dof_solution(store='dofsol')
r = structure.reaction_forces()
f = structure.internal_forces()


In [159]:
cd.db.K.to_numpy()[0]

array([[ 1.81629090e+08,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -2.07576103e+08,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.59470128e+07,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  3.18226990e+10,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  7.11198575e+11,
         0.00000000e+00, -2.23983804e+10,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.19991324e+12,
         0.00000000e+00, -9.42431856e+09,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.51239064e+11],
       [ 0.00000000e+00,  0.00000000e+00,  3.18226990e+10,
         0.00000000e+00, -7.11198575e+11,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -2.23983804e+10,
         0.00000000e+00, -1.19991324e+12,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -9.42431856e

In [160]:
ak.to_parquet(pd.db, 'pd.pa')
ak.to_parquet(cd.db, 'cd.pa')

In [161]:
A, Ix, Iy, Iz


(37067.16118306447, 2083182704.9778206, 1041591352.4889101, 1041591352.4889106)

In [162]:
u_x_exact = 0.5 * q * L**2 / (Ex * A)
u_x_exact

0.0004014591866763

In [163]:
d[iN, 0]


array([0.00040146, 0.        , 0.        ])

In [164]:
r[0, :, 0]

array([-2500.,     0.,     0.,     0.,     0.,     0.])

In [165]:
f[0, 0, :, 0]

array([2500.,    0.,    0.,    0.,    0.,    0.])

In [166]:
d[iN, 0]

array([0.00040146, 0.        , 0.        ])

In [167]:
cd.nodes

array([[ 0, 26,  1],
       [ 1, 27,  2],
       [ 2, 28,  3],
       [ 3, 29,  4],
       [ 4, 30,  5],
       [ 5, 31,  6],
       [ 6, 32,  7],
       [ 7, 33,  8],
       [ 8, 34,  9],
       [ 9, 35, 10],
       [10, 36, 11],
       [11, 37, 12],
       [12, 38, 13],
       [13, 39, 14],
       [14, 40, 15],
       [15, 41, 16],
       [16, 42, 17],
       [17, 43, 18],
       [18, 44, 19],
       [19, 45, 20],
       [20, 46, 21],
       [21, 47, 22],
       [22, 48, 23],
       [23, 49, 24],
       [24, 50, 25]], dtype=int64)